<a href="https://colab.research.google.com/github/priyanshsaxena24/AAD_Project/blob/main/ASDA_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --no-cache-dir --force-reinstall flwr tomli
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 127.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.0/540.0 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 150.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 137.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 124.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 170.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 106.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.0/763.0 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 127.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 106.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 363.4/363.4 MB 63.9 MB/s eta 0:00:01^C


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import copy

# Model definition
class IoTNet(nn.Module):
    def __init__(self, input_size=41):
        super(IoTNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# FGSM adversarial attack
def fgsm_attack(model, loss_fn, data, target, epsilon=0.05):
    data.requires_grad = True
    output = model(data)
    loss = loss_fn(output, target)
    loss.backward()
    return (data + epsilon * data.grad.sign()).detach()

# Simulate IoT data
def generate_data(samples=1000, features=41):
    X = np.random.rand(samples, features).astype(np.float32)
    y = (X[:, 0] + X[:, 1] * 0.5 > 0.75).astype(np.int64)
    return TensorDataset(torch.tensor(X), torch.tensor(y))

# Train a single client
def train_client(model, dataset, epochs=1):
    model_copy = copy.deepcopy(model)
    optimizer = torch.optim.Adam(model_copy.parameters(), lr=0.005)
    loader = DataLoader(dataset, batch_size=64, shuffle=True)
    loss_fn = nn.CrossEntropyLoss()

    model_copy.train()
    for _ in range(epochs):
        for x, y in loader:
            x_adv = fgsm_attack(model_copy, loss_fn, x.clone(), y)
            optimizer.zero_grad()
            loss = loss_fn(model_copy(x_adv), y)
            loss.backward()
            optimizer.step()
    return model_copy.state_dict()

# Federated averaging
def average_weights(weights_list):
    avg_weights = copy.deepcopy(weights_list[0])
    for key in avg_weights.keys():
        for w in weights_list[1:]:
            avg_weights[key] += w[key]
        avg_weights[key] = avg_weights[key] / len(weights_list)
    return avg_weights

# Evaluation function
def evaluate(model, dataset):
    model.eval()
    loader = DataLoader(dataset, batch_size=64)
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in loader:
            out = model(x)
            pred = out.argmax(dim=1)
            correct += (pred == y).sum().item()
            total += y.size(0)
    print(f"Test Accuracy: {correct / total:.2%}")

# Federated learning loop
global_model = IoTNet()
clients_data = [generate_data() for _ in range(3)]  # 3 simulated clients
rounds = 3

for r in range(rounds):
    print(f"\n--- Round {r+1} ---")
    client_weights = [train_client(global_model, data) for data in clients_data]
    averaged_weights = average_weights(client_weights)
    global_model.load_state_dict(averaged_weights)

# Final evaluation
test_data = generate_data(500)
evaluate(global_model, test_data)



--- Round 1 ---

--- Round 2 ---

--- Round 3 ---
Test Accuracy: 88.80%
